In [3]:
import json
import csv

In [4]:
with open('data/raw/worldbank-emissions.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        with open('data/emissions/' + row['Country Name'] + '.json', 'w') as f:
            json.dump({
                'x': list(range(1960, 2018)),
                'y': [row[str(i)] for i in range(1960, 2018)],
                'mode': 'lines+markers',
                'name': row['Country Name'] + "(Total CO2 Emissions in metric tons per capita)",
            }, f)

In [6]:
with open('data/raw/worldbank-emissions_total.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        with open('data/emissions_total/' + row['Country Name'] + '.json', 'w') as f:
            json.dump({
                'x': list(range(1960, 2018)),
                'y': [row[str(i)] for i in range(1960, 2018)],
                'mode': 'lines+markers',
                'name': row['Country Name'] + "(CO2 emissions in kt)",
            }, f)